Template list file

In [58]:
from astropy.table import Table, hstack
from pathlib import Path
import numpy as np
from eazy import filters

WD = Path.cwd()
TEMPDIR = WD.parent / "BROWN_COSMOS"
REDDENED_TEMPDIR = WD / "BROWN_COSMOS_REDDENED"
INPUTDIR = WD / "input"

INPUTDIR.mkdir(exist_ok=True)

temppaths_brown = list(TEMPDIR.glob("*.dat"))
temppaths_cosmos = list(TEMPDIR.glob("*.sed"))
temppaths = temppaths_brown + temppaths_cosmos

temppaths_reddened = list(REDDENED_TEMPDIR.glob("*.txt"))

Avs = [0.25, 0.5, 1.0]

In [3]:
# original Brown+COSMOS template
with open("BROWN_COSMOS.template", "w") as f:
    for i, temppath in enumerate(temppaths):
        f.write(f"{i+1:3d} {temppath}\n")

In [6]:
# with additional reddened templates
with open("BROWN_COSMOS+reddened.template", "w") as f:
    for i, temppath in enumerate(temppaths):
        f.write(f"{i+1:3d} {temppath}\n")
    for j, temppath_reddened in enumerate(temppaths_reddened):
        f.write(f"{i+j+2:3d} {temppath_reddened}\n")

Translate file

In [19]:
# make a dummy translate file, and make the column names of input catalogs correspond to the filter numbers
with open("dummy.translate", "w") as f:
    f.write("dummy F999")

Input catalogs

In [48]:
# first read filter numbers that correspond to 7DT filters
filt_latest_7dt = filters.FilterFile(WD / "FILTER.RES+7DT.res")
ids_7dt = filt_latest_7dt.search("7DT", verbose=False)
nums_7dt = ids_7dt + 1

flux_colnames = np.array([f"F{num}" for num in nums_7dt])
erro_colnames = np.array([f"E{num}" for num in nums_7dt])

In [55]:
filt_latest_7dt.filters[ids_7dt[0]].pivot

425.51512304174685

In [23]:
ids_7dt

array([418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430,
       431, 432, 433, 434, 435, 436, 437])

In [21]:
nums_7dt

array([419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431,
       432, 433, 434, 435, 436, 437, 438])

In [60]:
datatab = Table.read(WD / "Fiducial_fluxes_7DS_v2_more.fits")

cosmos2020 = Table.read(WD / "cosmos2020.fits")
cosmos2020.sort("Farmer")

cospd = cosmos2020.to_pandas()
datpd = datatab["source_id", "HSC_i_MAG", "lp_zBEST", "FLAG_ML"].to_pandas()
df = datpd.merge(
    cospd.rename(columns={"Farmer": "source_id", "RAJ2000": "ra", "DEJ2000": "dec"})
).rename(columns={"source_id": "ID", "lp_zBEST": "zpsec"})

base = Table.from_pandas(df)
base

ID,HSC_i_MAG,zpsec,FLAG_ML,ra,dec,E_B-V_,Classic,EZzphot
int64,float64,float64,float64,float64,float64,float64,int32,float64
12,23.990103981421147,0.707,0.0,150.47679031,2.33150699,0.014723000000000002,962809,0.6809810000000001
27,24.875468381594267,0.7751,0.0,150.71248825,1.79210579,0.018367,387749,0.7748600000000001
57,24.701907916699273,0.9672,0.0,150.71818794,1.79272545,0.018322,389282,1.1766590000000001
73,22.48269003598378,0.429,1.0,149.46802488,1.6256133,0.016032,223727,0.43394200000000005
74,23.767750415802425,1.5087,0.0,150.48801441,1.87542049,0.016019000000000002,474228,1.5545130000000003
82,24.707701668462914,0.3562,1.0,150.72868387,2.5438014,0.017903000000000002,1184955,0.39554600000000006
85,23.76854613020753,1.08,0.0,150.47643824,2.33418806,0.014780000000000001,965709,1.0853000000000002
104,24.480462347939646,1.2817,0.0,149.47572426,1.62630687,0.016092000000000002,223552,1.2979530000000001
129,24.334761096800534,3.4618,0.0,150.72468229,2.54390945,0.017883000000000003,1185951,3.1178930000000005


In [62]:
fluxlabels = [
    "fiducial_flux_RIS",
    "fiducial_flux_WFS",
    "fiducial_flux_IMS",
]
errolabels = [
    "fiducial_error_RIS",
    "fiducial_error_WFS",
    "fiducial_error_IMS",
]
surveylabels = ["RIS", "WFS", "IMS"]

for fl, el, sl in zip(fluxlabels, errolabels, surveylabels):
    fiducial_flux_array = np.array(
        [np.fromstring(row[1:-1], dtype=np.float64, sep=",") for row in datatab[fl]]
    )
    fiducial_erro_array = np.array(
        [np.fromstring(row[1:-1], dtype=np.float64, sep=",") for row in datatab[el]]
    )
    fluxtab = Table(data=fiducial_flux_array, names=flux_colnames)
    errotab = Table(data=fiducial_erro_array, names=erro_colnames)
    inputtab = hstack([base, fluxtab, errotab])
    inputtab.write(INPUTDIR / f"input_{sl}.csv", format="ascii.csv", overwrite=True)

In [64]:
inputtab

ID,HSC_i_MAG,zpsec,FLAG_ML,ra,dec,E_B-V_,Classic,EZzphot,F419,F420,F421,F422,F423,F424,F425,F426,F427,F428,F429,F430,F431,F432,F433,F434,F435,F436,F437,F438,E419,E420,E421,E422,E423,E424,E425,E426,E427,E428,E429,E430,E431,E432,E433,E434,E435,E436,E437,E438
int64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
12,23.990103981421147,0.707,0.0,150.47679031,2.33150699,0.014723000000000002,962809,0.6809810000000001,0.40049770149460284,0.5078999488291236,0.2963054331463698,0.3335773725294298,0.09632781075085359,0.21192752586334654,0.37209524197826066,0.29368319984589747,0.4985870900193809,0.4046981079202723,0.3725967722012696,0.6751451221323529,0.7335593340776831,1.0759871857767476,0.6807956725650958,1.184172133594535,0.9488228953306005,0.5163122176461388,1.267783441864066,1.0484799666863904,0.12772560052432094,0.11387962045581174,0.10839052197376808,0.10817636615811946,0.10437727725409991,0.1112288824337841,0.13267015359507206,0.13184519400400985,0.1461283265740689,0.17801847567707968,0.16864207043732382,0.1791378725006872,0.2072308907717925,0.2797763431091668,0.3134661478735132,0.4545645852922471,0.5045852615320934,0.5662332409277782,0.7237828478570381,1.0045318576971731
27,24.875468381594267,0.7751,0.0,150.71248825,1.79210579,0.018367,387749,0.7748600000000001,0.1555804207561737,0.23298046022847965,0.20890100061051156,0.2425297540900066,0.13264911732809592,-0.0433434970476219,0.35263735305899857,0.16517200662142031,0.3326562541108519,0.3481771377052659,0.34860829092313544,0.02851365349579364,0.008300258547255055,0.42246510939716175,0.46133413411519236,0.14976108017397355,0.851427575477548,-0.897450868898974,0.2007836151573084,2.190287917569994,0.12762509466835778,0.11378978870803297,0.10830610016743172,0.1081038162090855,0.10429418725856796,0.11114809639838144,0.1326006719991642,0.13176752195440508,0.14605500324842657,0.17786784737357897,0.16846603974830524,0.1789315807075524,0.20697474783076347,0.2795536598475472,0.3132480706679894,0.45435525020422407,0.5043646250919653,0.5659866828358857,0.7234516116407833,1.0043847645427422
57,24.701907916699273,0.9672,0.0,150.71818794,1.79272545,0.018322,389282,1.1766590000000001,-0.054737980225801494,0.04076737416359779,0.19915366253184433,0.03687395897230117,-0.09475717700582041,0.22444761604077712,0.30358720097369435,-0.06753550541133471,0.12421776007421409,0.17713751348287443,0.08627032336250276,0.2326219235041975,0.21709594958645737,0.07597485583211182,0.677247753679458,0.03490887039540123,-0.2480045975157707,0.7788508763142791,1.1624588195869474,1.2659804772021994,0.1275851634339583,0.11375130576334735,0.10826689082059997,0.1080594647816753,0.1042494542311209,0.11110165214751623,0.13255633589986265,0.13172823514975307,0.14601737853200192,0.17783937574050313,0.16842310562925578,0.17887109400639423,0.20692856795829775,0.2795355640202538,0.3132266091940865,0.45435071828231227,0.504491708276954,0.566135925331746,0.7235979580280588,1.0046030766284597
73,22.48269003598378,0.429,1.0,149.46802488,1.6256133,0.016032,223727,0.43394200000000005,0.7226129138786842,1.014288922428972,1.1581251416199758,1.3547618005051005,1.400602041888884,1.616632135249312,2.2694297446494174,2.474247850057344,2.9859002690302696,2.6522513206686162,3.3694931983719636,3.5530225794887795,3.283500613480419,3.7532825007462294,3.793716830437237,3.65305438092117,3.914739544396484,4.734975203242984,5.164469807302925,4.4082057736409705,0.12824829191612544,0.11452386712447703,0.10910970207477683,0.10900060223368876,0.10529780578041016,0.11253702196946067,0.1347654223654789,0.1348061563993731,0.14945706282396798,0.18067431455374316,0.17213943650547076,0.18271779798586546,0.21057968912922717,0.28269782

In [66]:
np.median(base['E_B-V_'])

0.016577

In [68]:
10**(0.4 * 0.002)

1.001843765724026